In [1]:
import pandas as pd
df = pd.read_csv("/content/100_Unique_QA_Dataset.csv")
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [6]:
# Tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace("?","")
  text = text.replace("'","")
  return text.split()

In [7]:
tokenize("what is the capital of india?")

['what', 'is', 'the', 'capital', 'of', 'india']

In [8]:
# Vocab
vocab  = {'<UNK>':0}


In [9]:
def build_vacab(row):
  tokenized_question = tokenize(row['question'])
  tokenize_answer = tokenize(row['answer'])
  merged_tokens = tokenized_question + tokenize_answer
  for token in merged_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)

In [10]:
df.apply(build_vacab,axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [12]:
len(vocab)

324

In [16]:
# Convert words to numerical indices
def text_to_indices(text,vocab):
  indexed_text = []
  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])
  return indexed_text

In [17]:
text_to_indices('Who is Vishwajeet',vocab)

[10, 2, 0]

In [19]:
import torch
from torch.utils.data import Dataset,DataLoader

In [20]:
class QADataset(Dataset):
  def __init__(self,df,vocab):
    self.df = df
    self.vocab = vocab
  def __len__(self):
    return self.df.shape[0]
  def __getitem__(self,index):
    numerical_question = text_to_indices(self.df.iloc[index]['question'],self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'],self.vocab)
    return torch.tensor(numerical_question),torch.tensor(numerical_answer)

In [21]:
dataset = QADataset(df,vocab)

In [22]:
dataloader = DataLoader(dataset,batch_size=1,shuffle=True)

In [23]:
for question,answer in dataloader:
  print(question,answer[0])

tensor([[  1,   2,   3,  33,  34,   5, 245]]) tensor([246])
tensor([[ 42, 312,   2, 313,  62,  63,   3, 314, 315]]) tensor([316])
tensor([[42, 18,  2, 62, 63,  3, 64, 18]]) tensor([65])
tensor([[ 1,  2,  3,  4,  5, 99]]) tensor([100])
tensor([[  1,   2,   3, 180, 181, 182, 183]]) tensor([184])
tensor([[  1,   2,   3,  37,  38,  39, 161]]) tensor([162])
tensor([[ 1,  2,  3, 24, 25,  5, 26, 19, 27]]) tensor([28])
tensor([[  1,   2,   3,   4,   5, 206]]) tensor([207])
tensor([[10, 55,  3, 56,  5, 57]]) tensor([58])
tensor([[ 1,  2,  3, 50, 51, 19,  3, 45]]) tensor([52])
tensor([[ 78,  79, 288,  81,  19,  14, 289]]) tensor([85])
tensor([[ 1,  2,  3, 59, 25,  5, 26, 19, 60]]) tensor([61])
tensor([[ 78,  79, 150, 151,  14, 152, 153]]) tensor([154])
tensor([[ 1,  2,  3, 69,  5, 53]]) tensor([260])
tensor([[1, 2, 3, 4, 5, 6]]) tensor([7])
tensor([[ 42, 167,   2,   3,  17, 168, 169]]) tensor([170])
tensor([[  1,   2,   3, 103,   5, 104,  19, 105]]) tensor([106])
tensor([[ 10,  11, 157, 158, 159

In [24]:
import torch.nn as nn

In [37]:
class SimpleRNN(nn.Module):
  def  __init__(self,vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size,embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64,vocab_size)

  def forward(self,question):
    embedded_question = self.embedding(question)
    hidden,final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))
    return output


In [38]:
x = nn.Embedding(324,embedding_dim=50)
y = nn.RNN(50,64,batch_first=True)
z = nn.Linear(64,324)

a = dataset[0][0].reshape(1,6)
print('Shape of a',a.shape)

b = x(a)
print('Shape of b',b.shape)

c,d = y(b)
print('Shape of c',c.shape)
print('Shape of d',d.shape)

e = z(d.squeeze(0))
print('Shape of e',e.shape)

Shape of a torch.Size([1, 6])
Shape of b torch.Size([1, 6, 50])
Shape of c torch.Size([1, 6, 64])
Shape of d torch.Size([1, 1, 64])
Shape of e torch.Size([1, 324])


In [39]:
learning_rate = 0.001
epochs = 20

In [40]:
model = SimpleRNN(len(vocab))

In [41]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [45]:
# Training loop
for epoch in range(epochs):
  total_loss = 0
  for question,answer in dataloader:
    optimizer.zero_grad()

    # Forward pass
    output = model(question)

    # Loss -> output shape(1,324) - (1)
    # Select the first token of the answer sequence as the target
    target = answer[:, 0] # Shape will be (batch_size,) which is (1,) for batch_size=1

    loss = criterion(output,target) # Pass the adjusted target

    # Gradients
    loss.backward()

    # update
    optimizer.step()

    # Correctly accumulate the total loss
    total_loss += loss.item()

  print(f"Epoch : {epoch + 1} , Loss : {total_loss:4f}")

Epoch : 1 , Loss : 10.083217
Epoch : 2 , Loss : 8.988297
Epoch : 3 , Loss : 7.944686
Epoch : 4 , Loss : 7.117677
Epoch : 5 , Loss : 6.360485
Epoch : 6 , Loss : 5.707084
Epoch : 7 , Loss : 5.201964
Epoch : 8 , Loss : 4.722355
Epoch : 9 , Loss : 4.314450
Epoch : 10 , Loss : 3.922393
Epoch : 11 , Loss : 3.596089
Epoch : 12 , Loss : 3.314453
Epoch : 13 , Loss : 3.068239
Epoch : 14 , Loss : 2.829581
Epoch : 15 , Loss : 2.622823
Epoch : 16 , Loss : 2.434852
Epoch : 17 , Loss : 2.265188
Epoch : 18 , Loss : 2.110116
Epoch : 19 , Loss : 1.970659
Epoch : 20 , Loss : 1.836641


In [46]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [47]:
predict(model, "What is the largest planet in our solar system?")

jupiter


In [48]:
predict(model, "whp is vk?")

I don't know
8
